In [36]:
from binance.client import Client
import datetime
from pytz import timezone
import os
import pandas as pd
from pandas import Series, DataFrame

In [6]:
client = Client('', '')

In [14]:
candles = client.get_klines(symbol='BNBBTC', interval=Client.KLINE_INTERVAL_1HOUR)
candles

[[1530867600000,
  '0.00204330',
  '0.00205150',
  '0.00202500',
  '0.00203180',
  '34380.24000000',
  1530871199999,
  '69.98056817',
  4535,
  '11998.44000000',
  '24.45334837',
  '0'],
 [1530871200000,
  '0.00203180',
  '0.00204120',
  '0.00202630',
  '0.00203190',
  '21444.30000000',
  1530874799999,
  '43.60021909',
  3045,
  '10111.41000000',
  '20.57020651',
  '0'],
 [1530874800000,
  '0.00203190',
  '0.00204170',
  '0.00203020',
  '0.00203700',
  '20007.53000000',
  1530878399999,
  '40.71737625',
  3155,
  '10631.05000000',
  '21.63534686',
  '0'],
 [1530878400000,
  '0.00203710',
  '0.00203890',
  '0.00202340',
  '0.00202750',
  '36141.25000000',
  1530881999999,
  '73.37120549',
  4948,
  '14591.88000000',
  '29.63729809',
  '0'],
 [1530882000000,
  '0.00202620',
  '0.00203990',
  '0.00202090',
  '0.00203650',
  '37203.33000000',
  1530885599999,
  '75.49115548',
  4155,
  '19581.66000000',
  '39.75278995',
  '0'],
 [1530885600000,
  '0.00203590',
  '0.00203800',
  '0.002025

In [182]:
klines = client.get_historical_klines("BNBBTC", 
                                      Client.KLINE_INTERVAL_1HOUR, 
                                      "2018/07/27 13:00:00 KST", 
                                      "2018/07/27 13:00:01 KST")
klines

[[1532664000000,
  '0.00168120',
  '0.00168160',
  '0.00166930',
  '0.00167860',
  '34756.00000000',
  1532667599999,
  '58.19191993',
  2566,
  '15669.95000000',
  '26.24728671',
  '0']]

In [50]:
print(type(klines))
print(type(klines[0][0]))
klines[0][0]

<class 'list'>
<class 'int'>


1532563200000

In [183]:
datetime.datetime.fromtimestamp(klines[0][0]/1000).strftime('%Y-%m-%d %H:%M:%S KST')

'2018-07-27 13:00:00 KST'

In [109]:
klines[14]

[1532667600000,
 '0.00167860',
 '0.00168940',
 '0.00167650',
 '0.00168210',
 '26905.42000000',
 1532671199999,
 '45.30756854',
 2100,
 '11022.13000000',
 '18.55915285',
 '0']

In [111]:
klines[14]

[1532667600000,
 '0.00167860',
 '0.00168940',
 '0.00167650',
 '0.00168160',
 '27522.32000000',
 1532671199999,
 '46.34475107',
 2153,
 '11309.51000000',
 '19.04259221',
 '0']

In [172]:
now_date = datetime.datetime.today()
now_date - datetime.timedelta(days=365)

datetime.datetime(2017, 7, 27, 15, 34, 51, 576513)

In [184]:
[1,2,3] + [2,4,4]

[1, 2, 3, 2, 4, 4]

In [66]:
HOUR_FORMAT = '%Y/%m/%d %H:%M:%S'

def to_date(date_str) :
    return datetime.datetime.strptime(date_str, HOUR_FORMAT)

def to_hour_str(timestamp) :
    if isinstance(timestamp, pd.Series) :
        return  pd.DatetimeIndex(pd.to_datetime(timestamp, unit='ms')).tz_localize('UTC').tz_convert('Asia/Seoul').strftime(HOUR_FORMAT)
    elif isinstance(timestamp, datetime.datetime) : 
        return timestamp.strftime(HOUR_FORMAT)
    else :
        return datetime.datetime.fromtimestamp(timestamp/1000).strftime(HOUR_FORMAT)

    
def get_klines_binance_data(symbol, start) :
    TZ_STR = ' KST'
    client = Client('', '')
    klines = []
    if start == '' :
        now_datetime = datetime.datetime.today()
        start = to_hour_str(now_datetime - datetime.timedelta(days=365)) 
        end = to_hour_str(now_datetime - datetime.timedelta(hours=1)) 
        print(end)
        while True :
            klines_new = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, start + TZ_STR, end + TZ_STR)
            if len(klines_new) == 0:
                break
            klines = klines_new +  klines   
            end = to_hour_str(to_date(start) - datetime.timedelta(hours=1))  
            start = to_hour_str(to_date(start) - datetime.timedelta(days=365))  
    else :    
        klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, start + TZ_STR)
        klines = klines[:-1]
    
    df_klines = pd.DataFrame(klines, columns=['open-time', 'open', 'high', 'low', 'close', 'volume', 
                                              'close-time', 'quote-volume', 'trades-cnt', 
                                              'taker-base-volume', 'taker-quote-volume', 'unknown'])   
    df_klines = df_klines.assign(hour = to_hour_str(df_klines['open-time']))
    df_klines = df_klines[['hour', 'open', 'high', 'low', 'close', 'volume']]
    return df_klines
    
def get_klines_data(symbol) :
    file_path = './data/' + symbol + '.csv'
    if os.path.isfile(file_path) :
        data = pd.read_csv(file_path)
        data = data[:-1]
        hours_last = data.tail(1)['hour'].to_string(index=False)
        hour_next = to_hour_str(to_date(hours_last) + datetime.timedelta(hours=1))
        new_data = get_klines_binance_data(symbol, hour_next)
        if len(new_data) > 0 :
            data = data.append(new_data, ignore_index=True)
            data.to_csv(file_path, index=False)
    else :
        data = get_klines_binance_data(symbol, '')
        data.to_csv(file_path, index=False)
    return data

#get_klines_data('BNBBTC')
#get_klines_binance_data('BNBBTC', '2018/08/07 11:00:00')

2018/08/07 12:15:36


,hour,open,high,low,close,volume
0,2017/07/14 13:00:00,0.00005000,0.00005480,0.00001000,0.00004955,6853648.00000000
1,2017/07/14 14:00:00,0.00004946,0.00004946,0.00003900,0.00004429,1734798.00000000
2,2017/07/14 15:00:00,0.00004429,0.00004890,0.00004361,0.00004800,998661.00000000
3,2017/07/14 16:00:00,0.00004799,0.00004841,0.00004700,0.00004780,537448.00000000
4,2017/07/14 17:00:00,0.00004780,0.00004790,0.00004331,0.00004410,249900.00000000
5,2017/07/14 18:00:00,0.00004411,0.00004678,0.00004282,0.00004282,183871.00000000
6,2017/07/14 19:00:00,0.00004282,0.00004500,0.00004215,0.00004490,202839.00000000
7,2017/07/14 20:00:00,0.00004490,0.00004800,0.00004450,0.00004537,420888.00000000
8,2017/07/14 21:00:00,0.00004406,0.00004536,0.00004360,0.00004362,147640.00000000
9,2017/07/14 22:00:00,0.00004361,0.00004700,0.00004358,0.00004400,208773.00000000


In [47]:
get_klines_binance_data('BNBBTC', '2018/08/07 11:00:00 KST')

,hour,open,high,low,close,volume
0,2018/08/07 02:00:00,0.00196180,0.00196770,0.00194540,0.00195340,59240.94000000


In [210]:
to_date('2018/07/27 15:00:00 KST')

datetime.datetime(2018, 7, 27, 15, 0)